<a href="https://colab.research.google.com/github/KanishkGar/Eva/blob/main/S5/Session_2_Step_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Code 3
Target: 
- The previous model was overfitting so I'll add dropout in the model
- To fix the overfitting, I'll also try to incorporate Data Transformations into the model.
- Parameters don't need to be reduced because I got them under 10000 in the last iteration

Results: 
- Parameters: 9762
- Best Test Accuracy: 99.0%
- Best Training Accuracy: 98.9%

Analysis:
- I just added dropout to the model and the model is now underfitting so I decided to not incorporate Data Transformations as well.
- Because the model is underfitting, that means I'm going to have to change the structure of the model rather than just add more regularization techniques.


In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
!/opt/bin/nvidia-smi
!nvcc --version

Sat Jan  2 18:10:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # input: 1x28x28
        # output: 6x14x14 
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, 3, padding=1),
            # RF - 3x3
            nn.ELU(),
            nn.BatchNorm2d(6),
            nn.Dropout(0.1),
            nn.Conv2d(6, 12, 3, padding=1),
            # RF - 5x5
            nn.ELU(),
            nn.BatchNorm2d(12),
            nn.Dropout(0.1),
            nn.Conv2d(12, 6, 1),
            nn.MaxPool2d(2, 2),
            # RF - 10x10
            nn.Dropout(0.1)
        )
        # input: 6x14x14
        # output: 12x7x7 
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 12, 3, padding=1),
            # RF - 12x12
            nn.ELU(),
            nn.BatchNorm2d(12),
            nn.Dropout(0.1),
            nn.Conv2d(12, 24, 3, padding=1),
            # RF - 14x14
            nn.ELU(),
            nn.BatchNorm2d(24),
            nn.Dropout(0.1),
            nn.Conv2d(24, 12, 1),
            nn.MaxPool2d(2, 2),
            # RF - 28x28
            nn.Dropout(0.1)
        )
        # input: 12x7x7
        # output: 24x1x1 
        self.conv3 = nn.Sequential(
            nn.Conv2d(12, 12, 3),
            # RF - 30x30
            nn.ELU(),
            nn.BatchNorm2d(12),
            nn.Conv2d(12, 12, 3),
            # RF - 32x32
            nn.ELU(),
            nn.BatchNorm2d(12),
            nn.Conv2d(12, 24, 3)
        )
      
                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = x.view(x.size(0), -1)
        return F.log_softmax(x)

In [9]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]              60
               ELU-2            [-1, 6, 28, 28]               0
       BatchNorm2d-3            [-1, 6, 28, 28]              12
           Dropout-4            [-1, 6, 28, 28]               0
            Conv2d-5           [-1, 12, 28, 28]             660
               ELU-6           [-1, 12, 28, 28]               0
       BatchNorm2d-7           [-1, 12, 28, 28]              24
           Dropout-8           [-1, 12, 28, 28]               0
            Conv2d-9            [-1, 6, 28, 28]              78
        MaxPool2d-10            [-1, 6, 14, 14]               0
          Dropout-11            [-1, 6, 14, 14]               0
           Conv2d-12           [-1, 12, 14, 14]             660
              ELU-13           [-1, 12, 14, 14]               0
      BatchNorm2d-14           [-1, 12,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [10]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        #add more transforms here later
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [11]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        #tqdm._instances.clear()



def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.08578453212976456 batch_id=468 Accuracy=92.05: 100%|██████████| 469/469 [00:14<00:00, 31.58it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0869, Accuracy: 9735/10000 (97%)

EPOCH: 2


loss=0.0596991591155529 batch_id=468 Accuracy=97.22: 100%|██████████| 469/469 [00:14<00:00, 31.71it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0268, Accuracy: 9907/10000 (99%)

EPOCH: 14


loss=0.02227902226150036 batch_id=468 Accuracy=98.91: 100%|██████████| 469/469 [00:14<00:00, 31.76it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0309, Accuracy: 9900/10000 (99%)

EPOCH: 15


loss=0.01774129644036293 batch_id=468 Accuracy=98.97: 100%|██████████| 469/469 [00:14<00:00, 32.08it/s]



Test set: Average loss: 0.0306, Accuracy: 9903/10000 (99%)

